In [1]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import os
path = 'gdrive/My Drive/Colab_Notebooks/cs224n-2019/a2'
os.chdir(path)

In [3]:
!dir

collect_submission.sh  __pycache__  Untitled0.ipynb  w2v-Skipgram.ipynb
env.yml		       run.py	    Untitled1.ipynb  word2vec.py
get_datasets.sh        sgd.py	    utils	     word_vectors.png


In [0]:
#!/usr/bin/env python

import numpy as np
import random

# from utils.gradcheck import gradcheck_naive
from utils.utils import normalizeRows, softmax


In [0]:
def sigmoid(x):
    """
    Compute the sigmoid function for the input here.
    Arguments:
    x -- A scalar or numpy array.
    Return:
    s -- sigmoid(x)
    """

    ### YOUR CODE HERE
    s = 1. / (1. + np.exp(-x))
    ### END YOUR CODE

    return s


- Steps for Naive Softmax Loss
  - Generate one-hot input vector $ x \in \mathbb{R}^{|V|}  $ of the center word.
  - Get our embedded word vector for center word $ v_{c} = Vx \in \mathbb{R}^{n}  $
  - Generate a score vector $ z = Uv_{c} $
  - Turn the score vector into probabilities, $ \hat{y} = softmax(z) $
  
  - $ P(O=0|C=c) = \hat{y} = {{\exp(u_{o}^{T}v_{c} )}\over{ \sum_{w \in Vocab} \exp(u_{w}^{T}v_{c} )}} $
  
- Naive Softmax Loss (Conditional Independence Assumption: for a single pair of words c and o)
  - $ \begin{matrix} J &=& - \log (P(O=0|C=c) ) \\
&=& - \sum_{w \in Vocab} y_{w}\log(\hat{y_{w}}) \\
&=& -\log(\hat{y_{o}}) \end{matrix} $
  
  

- Gradients
  - $ \begin{matrix} {{\partial{J}}\over{\partial{v_{c}}}} &=& U^{T}(\hat{y}-y) \end{matrix} $
  - $\begin{matrix} {{\partial{J}}\over{\partial{u_{w}}}} &=&  \begin{cases}
(\hat{y}-1)^{T}v_{c}, &  w=0 \\
(\hat{y})^{T}v_{c}, & w \neq 0 \end{cases}  \\ &=&  (\hat{y}-y)^{T}v_{c}
\end{matrix} $

In [0]:
def naiveSoftmaxLossAndGradient(
    centerWordVec,
    outsideWordIdx,
    outsideVectors,
    dataset
):
    """ Naive Softmax loss & gradient function for word2vec models
    Implement the naive softmax loss and gradients between a center word's
    embedding and an outside word's embedding. This will be the building block
    for our word2vec models.
    Arguments:
    centerWordVec -- numpy ndarray, center word's embedding
                    (v_c in the pdf handout)
    outsideWordIdx -- integer, the index of the outside word
                    (o of u_o in the pdf handout)
    outsideVectors -- outside vectors (rows of matrix) for all words in vocab
                      (U in the pdf handout)
    dataset -- needed for negative sampling, unused here.
    Return:
    loss -- naive softmax loss
    gradCenterVec -- the gradient with respect to the center word vector
                     (dJ / dv_c in the pdf handout)
    gradOutsideVecs -- the gradient with respect to all the outside word vectors
                    (dJ / dU)
    """

    ### YOUR CODE HERE

    ### Please use the provided softmax function (imported earlier in this file)
    ### This numerically stable implementation helps you avoid issues pertaining
    ### to integer overflow.
    gradCenterVec = np.zeros(outsideVectors.shape)

    z = np.dot(outsideVectors, centerWordVec)
    y_hat = softmax(z)
    loss = -np.log(y_hat[outsideWordIdx])
    
    y = np.zeros((outsideVectors.shape[0]))
    y[outsideWordIdx] = 1
    gradCenterVec = np.dot(outsideVectors.T,y_hat - y)
    #gradOutsideVecs = np.outer(y_hat - y, centerWordVec)
    gradOutsideVecs = np.dot((y_hat - y).reshape((outsideVectors.shape[0],1)), centerWordVec.reshape((1,outsideVectors.shape[1])))
    
    ### END YOUR CODE

    return loss, gradCenterVec, gradOutsideVecs

In [0]:
def getNegativeSamples(outsideWordIdx, dataset, K):
    """ Samples K indexes which are not the outsideWordIdx """

    negSampleWordIndices = [None] * K
    for k in range(K):
        newidx = dataset.sampleTokenIdx()
        while newidx == outsideWordIdx:
            newidx = dataset.sampleTokenIdx()
        negSampleWordIndices[k] = newidx
    return negSampleWordIndices


- Negative Sampling loss for a center word c, an outside word o, and K negative samples(10~15개가 적당하다고 함) from Vocab.
  - $J_{neg-sample} = -\log(\sigma(u_{o}^{T}v_{c})) - \sum_{k=1}^{K} \log(\sigma(-u_{k}^{T}v_{c})) $

  
- Gradients (using derivative of sigmoid)
  - $ \begin{matrix} {{\partial{J}}\over{\partial{v_{c}}}} &=& (\sigma(u_{o}^{T}v_{c})-1)u_{0} - \sum_{k=1}^{K}(\sigma(-u_{k}^{T}v_{c})-1)u_{k} \end{matrix}   $
  - $ \begin{matrix} {{\partial{J}}\over{\partial{u_{w}}}} &=&  \begin{cases}
(\sigma(u_{o}^{T}v_{c})-1)v_{c}, &  w = o \\
(1-\sigma(u_{k}^{T}v_{c}))v_{c}, & w = k & \mbox{here} \ k = 1 ,...,10 \end{cases}
\end{matrix}   $

In [0]:
def negSamplingLossAndGradient(
    centerWordVec,
    outsideWordIdx,
    outsideVectors,
    dataset,
    K=10
):
    """ Negative sampling loss function for word2vec models
    Implement the negative sampling loss and gradients for a centerWordVec
    and a outsideWordIdx word vector as a building block for word2vec
    models. K is the number of negative samples to take.
    Note: The same word may be negatively sampled multiple times. For
    example if an outside word is sampled twice, you shall have to
    double count the gradient with respect to this word. Thrice if
    it was sampled three times, and so forth.
    Arguments/Return Specifications: same as naiveSoftmaxLossAndGradient
    
    Arguments:
    centerWordVec -- numpy ndarray, center word's embedding
                    (v_c in the pdf handout)
    outsideWordIdx -- integer, the index of the outside word
                    (o of u_o in the pdf handout)
    outsideVectors -- outside vectors (rows of matrix) for all words in vocab
                      (U in the pdf handout)
    dataset -- needed for negative sampling, unused here.

    Return:
    loss -- naive softmax loss
    gradCenterVec -- the gradient with respect to the center word vector
                     (dJ / dv_c in the pdf handout)
    gradOutsideVecs -- the gradient with respect to all the outside word vectors
                    (dJ / dU)
    
    """

    # Negative sampling of words is done for you. Do not modify this if you
    # wish to match the autograder and receive points!
    negSampleWordIndices = getNegativeSamples(outsideWordIdx, dataset, K)
    indices = [outsideWordIdx] + negSampleWordIndices

    ### YOUR CODE HERE
    
    ### Please use your implementation of sigmoid in here.
    
    gradOutsideVecs = np.zeros(outsideVectors.shape)
    loss = 0

    z = sigmoid(np.dot(outsideVectors[outsideWordIdx], centerWordVec))

    loss += -np.log(z)
    gradOutsideVecs[outsideWordIdx] = np.dot(z-1.0, centerWordVec)
    gradCenterVec = np.dot(z - 1.0, outsideVectors[outsideWordIdx])

    for i in range(K):
        w_k = indices[i+1]
        z1 = sigmoid(-np.dot(outsideVectors[w_k], centerWordVec))
        loss += -np.log(z1)
        gradOutsideVecs[w_k] += np.dot(1.0 - z1, centerWordVec)
        gradCenterVec -= np.dot(z1 - 1.0, outsideVectors[w_k])

    ### END YOUR CODE

    return loss, gradCenterVec, gradOutsideVecs

 - $\begin{matrix} J 
&=& -\log P(w_{c-m},\dots,w_{c-1},w_{c+1},\dots,w_{c+m}|w_{c}) \\ 
&=& -\log \prod_{j=0,j \neq m}^{2m} P(w_{c-m+j}|w_{c}) \\
&=& -\log \prod_{j=0,j \neq m}^{2m} P(u_{c-m+j}|v_{c}) \\
&=& - \sum_{j=0,j \neq m}^{2m} \log P(u_{c-m+j}|v_{c}) \\
\end{matrix} $


- $  \partial J_{skipgram}(v_{c},w_{t-m},\dots,w_{t+m}, U)/\partial U  = \sum_{j=0,j \neq m} {{\partial J(v_{o},w_{w+j},U)}\over{\partial U}} $


- $ \partial J_{skipgram}(v_{c},w_{t-m},\dots,w_{t+m}, U)/\partial v_{w}  = \begin{cases} \sum_{j=0,j \neq m} {{\partial J(v_{o},w_{w+j},U)}\over{\partial v_{c}}} , & \mbox{when} \ w = c \\ 0 , & \mbox{when} \ w \neq c
\end{cases}  $


In [0]:
def skipgram(currentCenterWord, windowSize, outsideWords, word2Ind,
             centerWordVectors, outsideVectors, dataset,
             word2vecLossAndGradient=naiveSoftmaxLossAndGradient):
    """ Skip-gram model in word2vec
    Implement the skip-gram model in this function.
    Arguments:
    currentCenterWord -- a string of the current center word
    windowSize -- integer, context window size
    outsideWords -- list of no more than 2*windowSize strings, the outside words
    word2Ind -- a dictionary that maps words to their indices in
              the word vector list
    centerWordVectors -- center word vectors (as rows) for all words in vocab
                        (V in pdf handout)
    outsideVectors -- outside word vectors (as rows) for all words in vocab
                    (U in pdf handout)
    word2vecLossAndGradient -- the loss and gradient function for
                               a prediction vector given the outsideWordIdx
                               word vectors, could be one of the two
                               loss functions you implemented above.
    Return:
    loss -- the loss function value for the skip-gram model
            (J in the pdf handout)
    gradCenterVecs -- the gradient with respect to the center word vectors
            (dJ / dV in the pdf handout)
    gradOutsideVectors -- the gradient with respect to the outside word vectors
                        (dJ / dU in the pdf handout)
    """

    loss = 0.0
    gradCenterVecs = np.zeros(centerWordVectors.shape)
    gradOutsideVectors = np.zeros(outsideVectors.shape)

    ### YOUR CODE HERE
    center_word_indx = word2Ind[currentCenterWord]
    centerWordVec = centerWordVectors[center_word_indx]

    for w in outsideWords:
        _loss,_gradCenterVec, _gradOutsideVectors  = word2vecLossAndGradient(centerWordVec, word2Ind[w], outsideVectors, dataset)
        loss += _loss
        gradCenterVecs[center_word_indx] += _gradCenterVec
        gradOutsideVectors += _gradOutsideVectors
    ### END YOUR CODE

    return loss, gradCenterVecs, gradOutsideVectors


In [0]:
#############################################
# Testing functions below. DO NOT MODIFY!   #
#############################################

def word2vec_sgd_wrapper(word2vecModel, word2Ind, wordVectors, dataset,
                         windowSize,
                         word2vecLossAndGradient=naiveSoftmaxLossAndGradient):
    batchsize = 50
    loss = 0.0
    grad = np.zeros(wordVectors.shape)
    N = wordVectors.shape[0]
    centerWordVectors = wordVectors[:int(N/2),:] # 10 x 3행렬의 윗 부분 5 x 3을 centerWV로 지정
    outsideVectors = wordVectors[int(N/2):,:] # 나머지 아래부분을 outsideVector로 지정
    for i in range(batchsize):
        windowSize1 = random.randint(1, windowSize) # 1~5 randint
        # window size가 x면 context는 2배의 random한 a~e까지의 seq. centerword도 a~e 중 random
        centerWord, context = dataset.getRandomContext(windowSize1) 

        c, gin, gout = word2vecModel(
            centerWord, windowSize1, context, word2Ind, centerWordVectors,
            outsideVectors, dataset, word2vecLossAndGradient
        )
        loss += c / batchsize
        grad[:int(N/2), :] += gin / batchsize
        grad[int(N/2):, :] += gout / batchsize

    return loss, grad


In [0]:
def test_word2vec():
    """ Test the two word2vec implementations, before running on Stanford Sentiment Treebank """
    dataset = type('dummy', (), {})()
    def dummySampleTokenIdx():
        return random.randint(0, 4)

    def getRandomContext(C):
        tokens = ["a", "b", "c", "d", "e"]
        return tokens[random.randint(0,4)], \
            [tokens[random.randint(0,4)] for i in range(2*C)]
    dataset.sampleTokenIdx = dummySampleTokenIdx
    dataset.getRandomContext = getRandomContext

    random.seed(31415)
    np.random.seed(9265)
    dummy_vectors = normalizeRows(np.random.randn(10,3)) # 10 x 3 행렬, 정규화(행렬의 행값들 끼리 제곱해 더하여 루트 씌운 것으로 나눔)
    dummy_tokens = dict([("a",0), ("b",1), ("c",2),("d",3),("e",4)])

    print("==== Gradient check for skip-gram with naiveSoftmaxLossAndGradient ====")
    gradcheck_naive(lambda vec: word2vec_sgd_wrapper( # lambda의 vec은 dummy_vectors와 같은 값
        skipgram, dummy_tokens, vec, dataset, 5, naiveSoftmaxLossAndGradient),
        dummy_vectors, "naiveSoftmaxLossAndGradient Gradient")

    print("==== Gradient check for skip-gram with negSamplingLossAndGradient ====")
    gradcheck_naive(lambda vec: word2vec_sgd_wrapper(
        skipgram, dummy_tokens, vec, dataset, 5, negSamplingLossAndGradient),
        dummy_vectors, "negSamplingLossAndGradient Gradient")

    print("\n=== Results ===")
    print ("Skip-Gram with naiveSoftmaxLossAndGradient")

    print ("Your Result:")
    print("Loss: {}\nGradient wrt Center Vectors (dJ/dV):\n {}\nGradient wrt Outside Vectors (dJ/dU):\n {}\n".format(
            *skipgram("c", 3, ["a", "b", "e", "d", "b", "c"],
                dummy_tokens, dummy_vectors[:5,:], dummy_vectors[5:,:], dataset)
        )
    )

    print ("Expected Result: Value should approximate these:")
    print("""Loss: 11.16610900153398
Gradient wrt Center Vectors (dJ/dV):
 [[ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [-1.26947339 -1.36873189  2.45158957]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]
Gradient wrt Outside Vectors (dJ/dU):
 [[-0.41045956  0.18834851  1.43272264]
 [ 0.38202831 -0.17530219 -1.33348241]
 [ 0.07009355 -0.03216399 -0.24466386]
 [ 0.09472154 -0.04346509 -0.33062865]
 [-0.13638384  0.06258276  0.47605228]]
    """)

    print ("Skip-Gram with negSamplingLossAndGradient")   
    print ("Your Result:")
    print("Loss: {}\nGradient wrt Center Vectors (dJ/dV):\n {}\n Gradient wrt Outside Vectors (dJ/dU):\n {}\n".format(
        *skipgram("c", 1, ["a", "b"], dummy_tokens, dummy_vectors[:5,:],
            dummy_vectors[5:,:], dataset, negSamplingLossAndGradient)
        )
    )
    print ("Expected Result: Value should approximate these:")
    print("""Loss: 16.15119285363322
Gradient wrt Center Vectors (dJ/dV):
 [[ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [-4.54650789 -1.85942252  0.76397441]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]
 Gradient wrt Outside Vectors (dJ/dU):
 [[-0.69148188  0.31730185  2.41364029]
 [-0.22716495  0.10423969  0.79292674]
 [-0.45528438  0.20891737  1.58918512]
 [-0.31602611  0.14501561  1.10309954]
 [-0.80620296  0.36994417  2.81407799]]
    """)

In [0]:
# 원래 utils.gradcheck에 있던 것.

def gradcheck_naive(f, x, gradientText):
    """ Gradient check for a function f.
    Arguments:
    f -- a function that takes a single argument and outputs the
         loss and its gradients
    x -- the point (numpy array) to check the gradient at
    gradientText -- a string detailing some context about the gradient computation
    """
    print(f)

    rndstate = random.getstate()
    random.setstate(rndstate)
    fx, grad = f(x) # Evaluate function value at original point / word2vec_sgd_wrapper method로 이동
    h = 1e-4        # Do not change this!

    # Iterate over all indexes ix in x to check the gradient.
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        ix = it.multi_index
        x[ix] += h # increment by h
        random.setstate(rndstate)
        fxh, _ = f(x) # evalute f(x + h)
        x[ix] -= 2 * h # restore to previous value (very important!)
        random.setstate(rndstate)
        fxnh, _ = f(x)
        x[ix] += h
        numgrad = (fxh - fxnh) / 2 / h

        # Compare gradients
        reldiff = abs(numgrad - grad[ix]) / max(1, abs(numgrad), abs(grad[ix]))
        if reldiff > 1e-5:
            print("Gradient check failed for %s." % gradientText)
            print("First gradient error found at index %s in the vector of gradients" % str(ix))
            print("Your gradient: %f \t Numerical gradient: %f" % (
                grad[ix], numgrad))
            return

        it.iternext() # Step to next dimension

    print("Gradient check passed!")


In [13]:
#if __name__ == "__main__":
test_word2vec()


==== Gradient check for skip-gram with naiveSoftmaxLossAndGradient ====
<function test_word2vec.<locals>.<lambda> at 0x7f0c93937598>
Gradient check passed!
==== Gradient check for skip-gram with negSamplingLossAndGradient ====
<function test_word2vec.<locals>.<lambda> at 0x7f0c92e9c840>
Gradient check passed!

=== Results ===
Skip-Gram with naiveSoftmaxLossAndGradient
Your Result:
Loss: 11.16610900153398
Gradient wrt Center Vectors (dJ/dV):
 [[ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [-1.26947339 -1.36873189  2.45158957]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]
Gradient wrt Outside Vectors (dJ/dU):
 [[-0.41045956  0.18834851  1.43272264]
 [ 0.38202831 -0.17530219 -1.33348241]
 [ 0.07009355 -0.03216399 -0.24466386]
 [ 0.09472154 -0.04346509 -0.33062865]
 [-0.13638384  0.06258276  0.47605228]]

Expected Result: Value should approximate these:
Loss: 11.16610900153398
Gradient wrt Center Vectors (dJ/dV):
 [[ 0.      

## Reference 

- http://web.stanford.edu/class/cs224n/
- https://github.com/alongstar518/CS224NHomeworks/tree/master/homework2
- 224n 2017 assignment: https://dalpo0814.tistory.com/10
- Matrix calculus:  https://darkpgmr.tistory.com/141